### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [23]:
from sqp import *

TESTSTRING = "02002102000101000000102002110000021102201100000121101100102200102212002011112010100202020220021011110001020002102020211001102210020111001100000102100022100110201210022100020000101002210202100021000220220211202211110002221011010211000211202201021002102200012201101110222110002022012210202020020102100202211110202001122020000110020222220022110010020002102120002010010000211002021102102121210202221122000110202101020002020022200021000211020211022210200121022200010211002201101110220220110202110202210020212102102120002210002202112110210020001010002002000202102121222022121022201210211202020022100222101102112100221202021001010211020210102110202211200202000000000022102020000021111220012110201121010002002020000120200222022110202011002101002110010002120221100011000002100220222202021110222102200022001101011122021021111120021100010210222100222110202210102002221000021202020210200201101001120002211121011000212002000122022200121011120000210111011111020112221002002202"
TESTSTRING = TESTSTRING[:20]
numpy.printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [24]:
def run_sqp(test_string, lmbd):
    global pbm
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l), np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    # x0 = pbm.TransitionMatrix.reshape(-1)
    x0 = np.random.rand(n**2)
    x0 = np.hstack((x0, np.dot(C, x0))).copy()
    global res
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-8, 'disp': True, "maxiter":500},
            bounds=bounds, 
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [25]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.000)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)

[2024-02-06 05:03:05.485] Objective Fxn Value = 15.524919841234642
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 15.524919841234642
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
The original MLE estimated transition matrix is: 


/Users/hongdali/Desktop/repos/Operator-Lifting/src_python/sqp.py:216: RuntimeWarning: divide by zero encountered in scalar divide
  return x1/x2


array([[0.        , 1.        , 0.        ],
       [0.15384615, 0.61538462, 0.23076923],
       [0.33333333, 0.66666667, 0.        ]])

The New Transiiton Matrix after the SQP is: 


array([[0.08411812, 0.92080801, 0.45875018],
       [0.74091859, 0.42199995, 0.92126459],
       [0.54197698, 0.031475  , 0.32228115]])

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [26]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.5)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)


[2024-02-06 05:03:05.501] Objective Fxn Value = 19.13555976578798
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 19.13555976578798
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
The original MLE estimated transition matrix is: 


array([[0.        , 1.        , 0.        ],
       [0.15384615, 0.61538462, 0.23076923],
       [0.33333333, 0.66666667, 0.        ]])

The New Transiiton Matrix after the SQP is: 


array([[0.00766717, 0.32811193, 0.03268584],
       [0.09324308, 0.49884765, 0.57638589],
       [0.30565607, 0.71184944, 0.71684003]])

In [27]:
np.linalg.eig(np.dot(pbm.ConstraintMatrixC.transpose(), pbm.ConstraintMatrixC))

(array([1.99196035e+02, 1.28196558e+02, 6.70985579e+01, 3.32903608e+01,
        1.89796007e+01, 1.60292625e-14, 5.83064080e+00, 1.15763387e+01,
        1.89796007e+01]),
 array([[ 2.83803826e-04, -3.86296708e-02, -1.42183011e-01,
          3.53150451e-01,  8.16496581e-01, -3.33333333e-01,
         -2.08815589e-01, -1.79428789e-01,  2.48281085e-01],
        [-2.01887081e-03,  7.98151201e-05,  3.49254503e-04,
          7.27802998e-04, -1.96266703e-16, -3.33333333e-01,
          8.61735731e-01, -3.82486189e-01, -2.93817364e-16],
        [ 2.83803826e-04, -3.86296708e-02, -1.42183011e-01,
          3.53150451e-01, -4.08248290e-01, -3.33333333e-01,
         -2.08815589e-01, -1.79428789e-01, -7.97763120e-01],
        [ 6.01098185e-04,  6.85444694e-01,  5.78240892e-01,
         -1.82564334e-01, -2.69144149e-18, -3.33333333e-01,
         -1.86337143e-01, -1.28997299e-01,  1.14891261e-16],
        [-7.09420031e-01,  2.68352474e-04,  1.01663484e-02,
          7.59039505e-02, -3.56248421e-17, -3.